## Week3 HW 

### 오다건

In [58]:
#output size = (input size - filter size + (2*padding) / stride) +1
#filter는 작게, layer 깊게, fully-connected layer 조금만


## Q1

1. zero padding 크기 1만큼 적용
2. stride (1,1) 로 filter 적용
3. stride  (2,2)  로 2*2 maxpooling 적용했을 때 이미지 파일 어떻게 생겼는지 예측

In [4]:
#먼저 이미지 파일에 해당하는 데이터 만들어주기, filter 도 만들어주기

import numpy as np

image = np.array([[2,0,2,1,3,2],
                  [0,2,0,2,2,2],
                  [1,0,1,3,1,1],
                  [0,0,1,1,1,0],
                  [0,1,3,4,1,0],
                  [0,1,0,0,5,2]])

filter = np.array ([[1,0,0],
                    [0,1,0],
                    [0,0,1]])



In [6]:
image.shape, filter.shape

((6, 6), (3, 3))

In [11]:
#1. zero padding 크기 1만큼씩. 8*8 만들어짐

image_zero = np.array([[0,0,0,0,0,0,0,0],
                      [0,2,0,2,1,3,2,0],
                      [0,0,2,0,2,2,2,0],
                      [0,1,0,1,3,1,1,0],
                      [0,0,0,1,1,1,0,0],
                      [0,0,1,3,4,1,0,0],
                      [0,0,1,0,0,5,2,0],
                      [0,0,0,0,0,0,0,0]])

image_zero

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
image_filtered = np.array([[4,0,4,3,5,2],
                           [0,5,3,5,4,5],
                           [1,1,4,4,3,3],
                           [1,4,5,3,4,1],
                           [1,1,3,10,4,1],
                           [0,1,1,3,9,3]])

image_filtered
image_filtered.shape

#output image size = (6-3 +(2*1))/1 + 1 = 6 size 같음!

(6, 6)

In [17]:
#근데 여기서 maxpooling 한번 더..근데 stride 2로

image_max = np.array([[5,5,5],
                      [4,5,4],
                      [1,10,9]])
image_max

array([[ 5,  5,  5],
       [ 4,  5,  4],
       [ 1, 10,  9]])

## Q2

1. layer1 의 kernel size를 4로 늘리고, layer3의 stride를 2로 늘려라

In [62]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device= 'cuda' if torch.cuda.is_available() else 'cpu'
    #for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
        
#학습에 사용되는 parameters 설정

learning_rate = 0.001
trainig_epochs = 15
batch_size = 100

#데이터셋 가져오기 (MNISTdata)

mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train=True,
                         transform = transforms.ToTensor(),
                         download = True)

mnist_test = dsets.MNIST(root= 'MNIST_data/',
                        train= False,
                        transform = transforms.ToTensor(),
                        download = True)


#dataset loader

data_loader = torch.utils.data.DataLoader (dataset= mnist_train,
                                          batch_size = batch_size,
                                          shuffle =True,
                                          drop_last = True)



In [78]:
#기존 CNN model

class CNN_(torch.nn.Module):
    
    def __init__(self):
        
        super(CNN_,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(), 
                                         torch.nn.MaxPool2d(kernel_size=2,stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(self.fc1, torch.nn.ReLU(), torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias =True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
   
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [79]:
model = CNN_().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [80]:
total_batch = len(data_loader)
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

print('Learning Finished')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.18799977
[Epoch:    2] cost = 0.0510102436
[Epoch:    3] cost = 0.0353842825
[Epoch:    4] cost = 0.0288783591
[Epoch:    5] cost = 0.022384692
[Epoch:    6] cost = 0.021043662
[Epoch:    7] cost = 0.0153902723
[Epoch:    8] cost = 0.014692462
[Epoch:    9] cost = 0.0127259605
[Epoch:   10] cost = 0.0126365405
[Epoch:   11] cost = 0.0106430324
[Epoch:   12] cost = 0.00864581391
[Epoch:   13] cost = 0.00948478375
[Epoch:   14] cost = 0.00641647354
[Epoch:   15] cost = 0.00896859448
Learning Finished


In [74]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.9868999719619751


In [56]:
#과제 CNN model

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size = 4, stride=1, padding=1),
            
        #conv1d는 text-cnn, conv2d는 이미지 분류
        # input channel size = 1, (input image의 RGB depth/28*28이 1장..? 흑백이니까)
        # output_volume_size =32 (conv layer 거쳐서 몇장의 필터 만들고 싶은가)
        # kernel_size = 4, filter의 사이즈 정하기 
        # padding, stride
            
        #여기에서 fully connected layer로 전달하기 전에 Node 계산하기 위해 아까 공식에 대입
        
        torch.nn.ReLU(), #렐루 적용
        torch.nn.MaxPool2d(kernel_size=2, stride =2)) #맥스풀링
        
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size = 3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride =2))
        
        self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64,128,kernel_size = 3, stride=2, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride =2, padding =1))  
        
        self.fc1 = torch.nn.Linear(2*2*128,625,bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        #fully connected layer
        self.layer4 = torch.nn.Sequential(self.fc1,
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=1-self.keep_prob))
        
        
        self.fc2 = torch.nn.Linear(625,10,bias = True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
        #625개의 Input에서 10개의 output
        
    def forward(self,x):
        out= self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1) #flatten them for fc
        out = self.layer4(out)
        out = self.fc2(out)
        
        return out
                                    

In [50]:
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [51]:
total_batch = len(data_loader)
model.train()
print('Learning strated. It takes sometime')

for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished')

Learning strated. It takes sometime
[Epoch:    1] cost = 0.268052489
[Epoch:    2] cost = 0.067129679
[Epoch:    3] cost = 0.0465257205
[Epoch:    4] cost = 0.0357986614
[Epoch:    5] cost = 0.0288355462
[Epoch:    6] cost = 0.0255141947
[Epoch:    7] cost = 0.0202469993
[Epoch:    8] cost = 0.0197669417
[Epoch:    9] cost = 0.0156582035
[Epoch:   10] cost = 0.0142434053
[Epoch:   11] cost = 0.0146718984
[Epoch:   12] cost = 0.0119579313
[Epoch:   13] cost = 0.0101621673
[Epoch:   14] cost = 0.0107738338
[Epoch:   15] cost = 0.0102506001
Learning Finished


In [76]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1)== Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())

Accuracy:  0.98580002784729


2-2. layer 3까지 output size는 
(2,2) 

2-3. 
filter의 개수 (kernel 개수가 커졌으니까) cost는 기존 모델에 비해 상승하고, 
accuracy 도 더 증가할 것 같다.

2-4.cost 는  상승했는데 accuracy는 살짝 더 낮다.

